In [ ]:
%config Completer.use_jedi = False
import urllib
from IPython.display import Image, display, clear_output
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import json
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

sns.set_style('whitegrid')
import os
import h5py
import numpy as np
import pandas as pd
import pickle

from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras import applications
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.python.keras.regularizers import l2, l1
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Convolution2D, ZeroPadding2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, History


In [ ]:
# prepare data augmentation configuration
img_width, img_height = 150, 150
batch_size = 4

# directories path
location = 'data1a'
train_data_dir = location+'/training'
validation_data_dir = location+'/validation'

train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator= test_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_height, img_width),
                                                   batch_size=128,
                                                   class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                        target_size=(img_height, img_width),
                                                       batch_size=32,
                                                       class_mode='binary')
    

In [ ]:
model = Sequential([
    Convolution2D(128, (3,3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D(2,2),
    Convolution2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Convolution2D(32, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1, activation='sigmoid')

])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer = RMSprop(learning_rate=0.001),
                  metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 15
model.fit(train_generator, epochs=NUM_EPOCHS, validation_data=validation_generator,verbose=2)

In [ ]:
def car_damaged_or_whole(image_path, model):
    img = load_img(image_path, target_size=(img_width, img_height)) # this is a PIL image 
    x = img_to_array(img) # this is a Numpy array with shape (3, 256, 256)
    x = x.reshape((1,) + x.shape)/255 # this is a Numpy array with shape (1, 3, 256, 256)
    print(x.shape)
    pred = model.predict(x)
    print ("Validating that damage exists...")
    print (pred)
    if pred[0][0] <=.5:
        print ("Damaged")
    else:
        print ("No Damage")

In [ ]:
damaged_car_names = os.listdir(train_data_dir + '/00-damage')
whole_car_names = os.listdir(train_data_dir + '/01-whole')

In [ ]:
damage = '/00-damage/'
whole = '/01-whole/'
image_path = train_data_dir + whole + whole_car_names[15]


In [ ]:
# Test
car_damaged_or_whole('car-damaged.jpg', model)

In [ ]:
model.summary()

In [ ]:
model.save("damaged_or_whole_weights.h5")